In [47]:
BASE_FOLDER = '../../raw_domo_kb'
OUTPUT_FOLDER  ='../../nbs/knowledge_base/'


In [63]:
# !pip install pyyaml


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
from os import listdir
from os.path import isfile, join


def crawl_directory(folder_path):
  res = []
  for r, d, f in os.walk(folder_path):
    for file in f:
        if file == "process.json":
            res.append(os.path.join(r, file))
  return res


category_ls = crawl_directory(os.path.join(BASE_FOLDER, 'category'))
category_ls[0:5]

['../../raw_domo_kb\\category\\0TO5w000000ZamlGAC\\process.json',
 '../../raw_domo_kb\\category\\0TO5w000000ZammGAC\\process.json',
 '../../raw_domo_kb\\category\\0TO5w000000ZamnGAC\\process.json',
 '../../raw_domo_kb\\category\\0TO5w000000ZamoGAC\\process.json',
 '../../raw_domo_kb\\category\\0TO5w000000ZampGAC\\process.json']

what is the longest breadcrumb?

In [3]:
import json

category_objs = []
for file_path in category_ls:
    with open(file_path , 'r' , encoding='utf-8' ) as f:
        category_objs.append(json.loads(f.read()))
        

In [4]:
category_objs[0:5]

test_category = category_objs[0]

top_category = []
max_breadcrumb = None
for category in category_objs:
    if not max_breadcrumb or len(category['breadcrumbs']) > len(max_breadcrumb['breadcrumbs']):
        max_breadcrumb = category
    if len(category['breadcrumbs']) == 2:
        top_category.append(category)
        

In [5]:
top_category[0]

{'file_path': '../../raw_domo_kb/category/0TO5w000000ZamlGAC/index.html',
 'content': {'section-list': [{'text': 'implementing_sso',
    'url': '/s/topic/0TO5w000000ZanoGAC'},
   {'text': 'certifications', 'url': '/s/topic/0TO5w000000ZanGGAS'},
   {'text': 'domo_security_options', 'url': '/s/topic/0TO5w000000ZandGAC'},
   {'text': 'governance_tools', 'url': '/s/topic/0TO5w000000ZannGAC'},
   {'text': 'controlling_access_in_domo',
    'url': '/s/topic/0TO5w000000ZanMGAS'},
   {'text': 'specifying_company_settings',
    'url': '/s/topic/0TO5w000000ZaoGGAS'},
   {'text': 'admin_settings_overview', 'url': '/s/topic/0TO5w000000Zan4GAC'},
   {'text': 'approvals', 'url': '/s/topic/0TO5w000000Zan5GAC'},
   {'text': 'workflows', 'url': '/s/topic/0TO5w000000ZkAMGA0'}]},
 'title': 'Administrating Domo',
 'title_clean': 'administrating_domo',
 'description': 'Learn how to manage users and groups, control access to content, configure security settings, and more.',
 'breadcrumbs': [{'text': 'domo', 

In [42]:
import shutil

def reset_folder(folder_path):
      for r, d, f in os.walk(folder_path):
        for file_name in f:
            file_path = os.path.join(r, file_name) 
            os.remove(file_path)
        
        for directory in d:
            folder_path = os.path.join(r, directory) 
            shutil.rmtree(folder_path)
        

reset_folder('../../nbs/knowledge_base/')


In [61]:
def create_folder(folder_path):
    if os.path.exists(folder_path):
        return

    os.mkdir(folder_path)


def create_category(category_obj, base_output_folder, posts_folder="posts"):

    output_folder = os.path.join(
        base_output_folder, category_obj['title_clean'])
    create_folder(output_folder)

    create_folder(os.path.join(output_folder, posts_folder))

    with open('template_listing.txt', 'r', encoding='utf-8') as f:
        listing_template = f.read()

    listing_title = category['title']

    listing_template = listing_template.replace(
        "<<TITLE>>", listing_title).replace('<<FOLDER>>', f'"{posts_folder}"')
    listing_template = listing_template.replace(
        "<<SUBTITLE>>", category['description']) if category['description'] else listing_template.replace('subtitle: <<SUBTITLE>>', '')

    listing_path = os.path.join(output_folder, 'index.qmd')

    with open(listing_path, 'w', encoding='utf-8') as f:
        f.write(listing_template)

    return {"section": listing_title,
            'contents': listing_path.replace('\\', '/').split('/nbs/')[1]}

   
add_to_yaml = [create_category(
    category_obj, base_output_folder=OUTPUT_FOLDER) for category_obj in category_objs]


In [75]:
add_to_yaml

[{'text': 'Working With DataSets In Domo',
  'href': 'knowledge_base/administrating_domo/index.qmd'},
 {'text': 'Working With DataSets In Domo',
  'href': 'knowledge_base/connecting_data_to_domo/index.qmd'},
 {'text': 'Working With DataSets In Domo',
  'href': 'knowledge_base/consuming_data_in_domo/index.qmd'},
 {'text': 'Working With DataSets In Domo',
  'href': 'knowledge_base/creating_content_in_domo/index.qmd'},
 {'text': 'Working With DataSets In Domo',
  'href': 'knowledge_base/creating_custom_visualizations/index.qmd'},
 {'text': 'Working With DataSets In Domo',
  'href': 'knowledge_base/distributing_domo_content/index.qmd'},
 {'text': 'Working With DataSets In Domo',
  'href': 'knowledge_base/getting_to_know_domo/index.qmd'},
 {'text': 'Working With DataSets In Domo',
  'href': 'knowledge_base/release_notes/index.qmd'},
 {'text': 'Working With DataSets In Domo',
  'href': 'knowledge_base/training/index.qmd'},
 {'text': 'Working With DataSets In Domo',
  'href': 'knowledge_base/

In [79]:
import yaml


def update_yaml():
    yaml_path = '../../nbs/_quarto.yml'

    with open(yaml_path, 'r') as f:
        quarto_yaml = yaml.safe_load(f)

    from pprint import pprint
    pprint(quarto_yaml)

    quarto_yaml['website']['sidebar'].update({'contents': [
        {'auto': "index.ipynb"},
        * add_to_yaml
    ]})
    return quarto_yaml

    # with open(yaml_path, 'w') as f:
    #     yaml.dump(quarto_yaml, f)


update_yaml()


{'format': {'html': {'css': 'styles.css', 'theme': 'cosmo', 'toc': True}},
 'metadata-files': ['nbdev.yml'],
 'project': {'type': 'website'},
 'website': {'navbar': {'background': 'primary', 'search': True},
             'open-graph': True,
             'repo-actions': ['issue'],
             'sidebar': {'contents': [{'auto': 'index.ipynb'},
                                      {'contents': 'test/*.ipynb',
                                       'section': 'Test'}],
                         'style': 'floating'},
             'twitter-card': True}}


{'project': {'type': 'website'},
 'format': {'html': {'theme': 'cosmo', 'css': 'styles.css', 'toc': True}},
 'website': {'twitter-card': True,
  'open-graph': True,
  'repo-actions': ['issue'],
  'navbar': {'background': 'primary', 'search': True},
  'sidebar': {'style': 'floating',
   'contents': [{'auto': 'index.ipynb'},
    {'text': 'Working With DataSets In Domo',
     'href': 'knowledge_base/administrating_domo/index.qmd'},
    {'text': 'Working With DataSets In Domo',
     'href': 'knowledge_base/connecting_data_to_domo/index.qmd'},
    {'text': 'Working With DataSets In Domo',
     'href': 'knowledge_base/consuming_data_in_domo/index.qmd'},
    {'text': 'Working With DataSets In Domo',
     'href': 'knowledge_base/creating_content_in_domo/index.qmd'},
    {'text': 'Working With DataSets In Domo',
     'href': 'knowledge_base/creating_custom_visualizations/index.qmd'},
    {'text': 'Working With DataSets In Domo',
     'href': 'knowledge_base/distributing_domo_content/index.qmd'}